In [11]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os


In [12]:
df = pd.read_csv("../data/res/all_results.csv", sep="\t")
print(len(df.index))
df.head(2)

270


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,24,seqname,position,start,end,MITE,mirbase,annotation,NEW_NAME,genes
0,1A:117385432-117385825,sun_mites_MITE_T_430_1A_117385432_117385825,394,17,0.135,3,0.941,+,UCUGCACCCUGAAUGAUGAAUAGU,11,...,14,1A,117385432-117385825,117385432,117385825,MITE_524,NaN,intergenic,miRNA_MITE_1,NaN
1,1A:169645857-169646029,mrcv_mites_MITE_T_62663_1A_169645857_169646029,173,16,0.126,3,1.000,+,UAGAGAUUUCAAAUGGAACAC,7,...,3,1A,169645857-169646029,169645857,169646029,DTT_Tdur_Thalos_103H9-1,NaN,intron,miRNA_MITE_2,TraesCS1A02G130800.1


In [13]:
df = df[df.MITE.notnull()]
print(len(df.index))
df.head(2)

38


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,24,seqname,position,start,end,MITE,mirbase,annotation,NEW_NAME,genes
0,1A:117385432-117385825,sun_mites_MITE_T_430_1A_117385432_117385825,394,17,0.135,3,0.941,+,UCUGCACCCUGAAUGAUGAAUAGU,11,...,14,1A,117385432-117385825,117385432,117385825,MITE_524,NaN,intergenic,miRNA_MITE_1,NaN
1,1A:169645857-169646029,mrcv_mites_MITE_T_62663_1A_169645857_169646029,173,16,0.126,3,1.000,+,UAGAGAUUUCAAAUGGAACAC,7,...,3,1A,169645857-169646029,169645857,169646029,DTT_Tdur_Thalos_103H9-1,NaN,intron,miRNA_MITE_2,TraesCS1A02G130800.1


In [14]:
# load all mites
fasta_seq = SeqIO.parse("../data/res/mites_consensus.fasta", 'fasta')
mites = {}
for record in fasta_seq:
    mites[record.id] = record.seq
    
# load all premirna
fasta_seq = SeqIO.parse("../data/res/all_results.fasta", 'fasta')
mirnas = {}
for record in fasta_seq:
    mirnas[record.id] = record.seq

In [15]:
from Bio.Align.Applications import MafftCommandline
import tempfile


In [17]:
to_write = ""
for k,v in df.iterrows():
    mirna = v['Name']
    mite =  v['MITE']
    newname = v['NEW_NAME']
    with tempfile.NamedTemporaryFile(mode = "w") as temp:
        query = f">{newname} {mirna}\n{str(mirnas[mirna])}"
        target = f">{mite}\n{str(mites[mite])}"
        temp.write(f"{query}\n{target}")
        temp.seek(0)
        mafft_cline = MafftCommandline(input=temp.name)
        mafft_cline.reorder = True
        mafft_cline.clustalout = True
        mafft_cline.auto = True
        stdout,stderr=mafft_cline()
        #fold 
        """with tempfile.NamedTemporaryFile(mode = "w") as tempfilemirna:
            query = f">{newname} {mirna}\n{str(mirnas[mirna])}"
            tempfilemirna.write(f"{query}")
            tempfilemirna.seek(0)
            dirname =  f"/home/juan/Documents/dev/bio/mirna_mite/data/res/rnaplots/{mirna}"
            os.system(f"mkdir -p {dirname}")
            cmd= f"cd {dirname} && RNAfold < {tempfilemirna.name}"
            #print(cmd)
            stream = os.popen(cmd)
            output = stream.readlines()
            os.popen(f" echo \"{output}\" > {dirname}/out.txt")
            #print(output)
        """
    to_write += f"{query}\n"
    to_write += f"{target}\n"
    to_write += f"\n-----\n"
    to_write += f"{stdout}"
    to_write += f"-----\n\n"
aln = open('../data/res/mite_micro_aln.fasta', 'w')
aln.write(to_write)

43417